In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

for handler in logging.root.handlers:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

In [3]:
# Set project root
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
PROJECT_ROOT_LS = [p.name for p in PROJECT_ROOT.iterdir()]
assert "featurologists" in PROJECT_ROOT_LS, f"Not a project root? {PROJECT_ROOT}, pwd: {Path().resolve()}"

In [4]:
import os

K8S_FEAST_NS = os.environ.get('K8S_FEAST_NS', 'feast-dev')
print(f'K8S_FEAST_NS={K8S_FEAST_NS}')

K8S_FEAST_NS=feast-dev


In [5]:
from typing import Tuple
import subprocess
import socketserver

def get_free_port():
    with socketserver.TCPServer(("localhost", 0), None) as s:
        free_port = s.server_address[1]
        return free_port

def kubectl_port_forward(ns: str, what: str, port_remote: int) -> Tuple[subprocess.Popen, int]:
    port_local = get_free_port()
    p = subprocess.Popen(
        ['kubectl', '-n', 'feast-dev', 'port-forward', what, f'{port_local}:{port_remote}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    print(f'pid: {p.pid}')
    try:
        n_sec = 2
        stdout, stderr = p.communicate(timeout=n_sec)
        raise ValueError(
            f'Port-forward process for {what} has terminated within {n_sec} sec: '
            f'stdout={stdout}, stderr={stderr}'
        )
    except subprocess.TimeoutExpired:
        print(f"Port-forward process for '{what}' seems to be working: check 'localhost:{port_local}'")
    
    return p, port_local

In [6]:
port_forward_processes = []

In [7]:
feast_core_process, feast_core_port = kubectl_port_forward(K8S_FEAST_NS, 'service/feast-release-feast-core', 6565)

port_forward_processes.append(feast_core_process)

pid: 3668491
Port-forward process for 'service/feast-release-feast-core' seems to be working: check 'localhost:53707'


In [8]:
import os

from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat

In [9]:
client = Client(core_url=f"localhost:{feast_core_port}") #, serving_url='localhost:8092')
client.list_projects()

/plain/github/opensource/Featurologists/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['default']

In [11]:
DATA_FEAST_PATH = PROJECT_ROOT / 'data-feast'

data_location = f'file://{DATA_FEAST_PATH}'

In [ ]:
driver_id = Entity(name="driver_id", description="Driver identifier", value_type=ValueType.INT64)

driver_statistics_source_uri = os.path.join(demo_data_location, "driver_statistics")

driver_statistics = FeatureTable(
    name = "driver_statistics",
    entities = ["driver_id"],
    features = [
        acc_rate,
        conv_rate,
        avg_daily_trips
    ],
    batch_source=FileSource(
        event_timestamp_column="datetime",
        created_timestamp_column="created",
        file_format=ParquetFormat(),
        file_url=driver_statistics_source_uri,
        date_partition_column="date"
    )
)

In [68]:
for p in port_forward_processes:
    p.kill()

/plain/github/opensource/Featurologists/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
